In [9]:
!pwd

/Users/irubachev/Documents/CS/cvision/03-sign-classification


In [10]:
import numpy as np
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage.io import imread
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score

from os.path import join

from ipywidgets import FloatProgress
from IPython.display import display

from time import time

import warnings
warnings.simplefilter('ignore')

In [11]:
def crop_and_rescale(im, size=(64,64)):
    im = rgb2gray(im)
    dim = max(im.shape)
    return resize(im[:dim,:dim], size, mode='constant')

In [14]:
def extract_hog(im, bins_count=9, cell_size=(8, 8), block_size=(2, 2), eps=1e-25):
    im = crop_and_rescale(im)
    im = np.sqrt(im)
    dy, dx = np.gradient(im, .5)
    
    mag = np.hypot(dx, dy)
    ang = np.rad2deg(np.arctan2(dy, dx)) % 180
    
    cell_x, cell_y = cell_size
    block_x, block_y = block_size

    count_cx = im.shape[1] // cell_x
    count_cy = im.shape[0] // cell_y

    histograms = np.zeros((count_cy, count_cx, bins_count))

    for y in range(count_cy):
        for x in range(count_cx):
            histograms[y, x] = make_histogram(mag, ang, cell_x, cell_y, cell_x * x, cell_y * y, bins_count)


    count_bx = count_cx - block_x + 1
    count_by = count_cy - block_y + 1
    normalized = np.zeros((count_by, count_bx, block_y, block_x, 9))
    for y in range(count_by):
        for x in range(count_bx):
            block = histograms[y:y + block_y, x:x + block_x, :]
            normalized[y, x, :] = block / np.sqrt(np.sum(block ** 2) + eps)

    return normalized.ravel()

In [15]:
def make_histogram(mag, ang, cell_x, cell_y, pos_x, pos_y, bins_count):
    hist_vec = np.zeros((bins_count,))
    for y in range(pos_y, pos_y + cell_y):
        for x in range(pos_x, pos_x + cell_x):
            cur_ang = ang[y, x]
            ind = int(cur_ang // (180 / bins_count))
            cur_ang %= (180 / bins_count)
            hist_vec[(ind + 1) % bins_count] += (cur_ang / 20) * mag[y, x]
            hist_vec[ind % bins_count] += (1. - cur_ang / 20) * mag[y, x]
    return hist_vec

In [16]:
def preprocess_data(data_dir, filename):
    fgt = open(join(data_dir, filename))
    next(fgt)
    lines = fgt.readlines()

    filenames = []
    labels = np.zeros(len(lines))
    for i, line in enumerate(lines):
        filename, label = line.rstrip('\n').split(',')
        filenames.append(filename)
        labels[i] = int(label)

    return filenames, labels

In [17]:
def extract_features(path, filenames):
    hog_length = len(extract_hog(imread(join(path, filenames[0]), plugin='matplotlib')))
    data = np.zeros((len(filenames), hog_length))
    
    f = FloatProgress(min=0, max=len(filenames)) # instantiate the bar
    display(f)
    
    for i in range(0, len(filenames)):
        f.value += 1
        
        filename = join(path, filenames[i])
        data[i, :] = extract_hog(imread(filename, plugin='matplotlib'))
    return data

In [21]:
def fit_and_classify(train_featues, train_labels, test_features):
    clf = svm.SVC(kernel='linear')
    clf.fit(train_featues, train_labels)
    return clf.predict(test_features), clf

In [19]:
%%time
data_path = "/Users/irubachev/Documents/CS/cvision/03-sign-classification/data/train"
data_filename = "gt.csv"

filenames, labels = preprocess_data(data_path, data_filename)

data = extract_features(data_path, filenames)

CPU times: user 9min 33s, sys: 23.5 s, total: 9min 56s
Wall time: 10min 8s


In [20]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [23]:
y_predicted, model = fit_and_classify(X_train, y_train, X_test)

In [25]:
print(model.score(X_test, y_test))

0.981764855904


In [15]:
scores = cross_val_score(model, data, labels, cv=5, verbose=1, n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 20.2min finished


In [16]:
np.mean(scores)

0.93172459675062524

In [34]:
linear_model = svm.SVC(kernel='linear', C=500)
linear_model.fit(X_train, y_train)

SVC(C=500, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [35]:
linear_model.score(X_train, y_train)

1.0

In [36]:
linear_model.score(X_test, y_test)

0.97908696761030345

In [17]:
scores

array([ 0.92935476,  0.93611324,  0.93649579,  0.93522061,  0.92143859])

C = $150$, kernel is rbf score $= 0,981828615149$